# change and save images

most important: 
* cv2.imread(..., cv.IMREAD_GRAYSCALE)

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

image_dir = r"J:\SelfStudying\MachineLearning\DeepLearningBasic\images"
CATEGORIES = ["daisy", "dandelion"]
IMAGE_SIZE = 50
train_data = []

for c in CATEGORIES: 
    c_dir = os.path.join(image_dir, c)
    for image_name in os.listdir(c_dir):
        image_path = os.path.join(c_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#         plt.imshow(image)
#         plt.show()
#         break
        array = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        index = CATEGORIES.index(c)
        train_data.append([array, index])

import random
random.shuffle(train_data)

import pickle
X = []
y = []
for array, index in train_data:
    X.append(array)
    y.append(y)
X = np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
with open("X.pickle", "wb") as f1:
    pickle.dump(X, f1)
with open("y.pickle", "wb") as f2:
    pickle.dump(y, f2)

# build and save models

In [24]:
import pickle
with open("X.pickle", "rb") as f:
    X = pickle.load(f)
with open("y.pickle", "rb") as f:
    y = pickle.load(f)

X = X/255.0

In [25]:
X.shape

(1531, 50, 50, 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

NAME = "2conv-1flatten-1dense"

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Flatten())

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation("sigmoid"))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss= "binary_crossentropy", 
            optimizer= "adam", 
              metrics= ["accuracy"], 
              callbacks= [tensorboard]
             )

model.fit(X, y, batch_size= 32, epochs= 3, validation_split= 0.3)

model.save("CNN.model")

# inference with new images

In [ ]:
image_infer_path = "infer_images.jpg"

def prepare(path):
    array = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(array, (IMAGE_SIZE, IMAGE_SIZE))
    return new_array.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)

model = tf.keras.models.load_model("CNN.model")
r = model.predict([prepare(image_infer_path)])
print(CATEGORIES[int(r)])